In [11]:
#imports
import datetime
import linecache

#initial information
beginingDate = ["8/6/2019"]
beginingTime = ["12:48:16 PM"]
results_folder = "/Users/user/projects/allee_effect/data/"
file_name = results_folder + "07082019_ClearSeal_copy.txt"

#put intoData
start_time = datetime.datetime(2019,8,6,12,48,16)
# (year, month, day, hour(military), minute, second)
date_line = "8/7/2019"
time_line = "10:17:30 AM"

def convert_to_timeDate (date, time):
    #getting the date
    first_dash = date.find("/")
    month = int(date[first_dash- 1: first_dash])
    half_date = date[first_dash + 1: len(date)]
    second_dash = half_date.find("/")
    day = int(half_date[0: second_dash])
    year = int(half_date[second_dash + 1: len(half_date)])

    #getting the time
    first_colon = time.find(":")
    initial_hour = int(time[0 : first_colon])
    half_time = time[first_colon + 1: len(time)]
    second_colon = half_time.find(":")
    minute = int(half_time[0: second_colon])
    space = half_time.find(" ")
    second = int(half_time[second_colon + 1: space])
    night = half_time[space + 1: len(half_time)]
    if night == "AM":
        if initial_hour == 12:
            hour = 0
        else:
            hour = initial_hour
    elif night == "PM":
        if initial_hour == 12:
            hour = 12
        else:
            hour = initial_hour + 12

    #get timeDate
    timeDate = datetime.datetime(year, month, day, hour, minute, second)
    return timeDate


def time_diff_hours (date, time, start_time):
    
    current_time = convert_to_timeDate(date,time)
    elapsed_time = current_time - start_time
    
    #changing elapsed time into hours
    seconds = elapsed_time.total_seconds()
    hours = round((seconds /60 /60),2)
    
    return hours


def find_initial_time (dateTime_matrix):
    times = []
    i = 0;
    for DandT in dateTime_matrix:
        date = DandT[0]
        time = DandT[1]
        date_time = convert_to_timeDate(date, time)
        times.append(date_time)
        i = i + 1
    
    first = times[0]
    for DT in times:
        if DT < first:
            first = DT
            
    return first
        
    
print (find_initial_time(dateTime))

2019-08-06 12:48:16


In [1]:
import sys, os, datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

data_directory = '/Users/user/projects/allee_effect/data'

def findVals(path, df=None):
    dfStart = 'Results'
    dfDate = ''
    dfTime = ''
    with open(path, 'rt', encoding = 'utf-8', errors='replace') as f:
        if df is None:
            for num, line in enumerate(f, 1):
                if dfStart in line:
                    return num
        for line in f:
            if 'Date' in line:
                dfDate = line.split('\t')[-1].strip('\n')
            if 'Time' in line:
                dfTime = line.split('\t')[-1].strip('\n')
    return dfDate, dfTime

def addMultiIndex(df, date, time):
    tempDf = df.copy()
    tempColumns = tempDf.columns
    dt = ', '.join((date, time))
    
    newColumns = []
    for i in tempColumns:
        if i <= 6:
            newColumns.append((dt, 'MR', i))
        elif i > 6:
            newColumns.append((dt, 'MVF', i))
    return newColumns
    
# def time_diff_hours (dateTime):
#     dt = 
    
#     #calculating elapsed time
#     time2 = datetime.datetime(year, month, day, hour, second, milisecond)
#     elapsed_time = time2 - start_time
    
#     #changing elapsed time into hours
#     seconds = elapsed_time.total_seconds()
#     hours = round((seconds /60 /60),2)
    
#     return hours
    
def preprocess(data_directory):
    reads_dfs = []
    dateTime = []

    for dirName, subdirList, fileList in os.walk(data_directory):
        for fname in fileList:
            extension = fname.split('.')[-1]
            if extension == 'txt':
                path = os.path.join(data_directory, fname)

                df = pd.read_csv(path, skiprows=findVals(path), sep='\t', header=0, index_col=0)
                df.columns = (range(1, df.shape[1]+1))
                df = df.drop(df.columns[-1], axis=1)

                date, time = findVals(path, df)
                dateTime.append((date, time))

                #addMultiIndex(df)
                df.columns = pd.MultiIndex.from_tuples(addMultiIndex(df, date, time))
                reads_dfs.append(df)

    odVals = pd.concat(reads_dfs, axis=1)
    print(dateTime)
    
    return odVals, dateTime

odVals, dateTime = preprocess(data_directory)

[('8/7/2019', '10:17:30 AM'), ('8/6/2019', '12:48:16 PM')]
